In [1]:
import pandas as pd
from colorama import Fore, Back, Style
import folium
from folium.plugins import MarkerCluster

In [2]:
compteur_file_path = '../data/comptage-velo-donnees-compteurs.csv'

In [3]:
df = pd.read_csv(compteur_file_path, sep = ';')
df

,id_compteur,nom_compteur,id,name,sum_counts,date,installation_date,url_photos_n1,coordinates,counter,photos,test_lien_vers_photos_du_site_de_comptage_,id_photo_1,url_sites,type_dimage,mois_annee_comptage
0,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,0.0,2021-12-01T03:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/10/6d...,"48.83504,2.33314",X2H20012081,https://filer.eco-counter-tools.com/file/10/6d...,https://filer.eco-counter-tools.com/file/10/6d...,https:,https://www.eco-visio.net/Photos/100003096,jpg,2021-12
1,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,2.0,2021-12-01T04:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/10/6d...,"48.83504,2.33314",X2H20012081,https://filer.eco-counter-tools.com/file/10/6d...,https://filer.eco-counter-tools.com/file/10/6d...,https:,https://www.eco-visio.net/Photos/100003096,jpg,2021-12
2,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,0.0,2021-12-01T05:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/10/6d...,"48.83504,2.33314",X2H20012081,https://filer.eco-counter-tools.com/file/10/6d...,https://filer.eco-counter-tools.com/file/10/6d...,https:,https://www.eco-visio.net/Photos/100003096,jpg,2021-12
3,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,213.0,2021-12-01T08:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/10/6d...,"48.83504,2.33314",X2H20012081,https://filer.eco-counter-tools.com/file/10/6d...,https://filer.eco-counter-tools.com/file/10/6d...,https:,https://www.eco-visio.net/Photos/100003096,jpg,2021-12
4,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,84.0,2021-12-01T07:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/10/6d...,"48.83504,2.33314",X2H20012081,https://filer.eco-counter-tools.com/file/10/6d...,https://filer.eco-counter-tools.com/file/10/6d...,https:,https://www.eco-visio.net/Photos/100003096,jpg,2021-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893091,300021320-353293614,69 Boulevard Ornano N-S,300021320,69 Boulevard Ornano,0.0,2023-01-07T05:00:00+01:00,2021-11-30,https://filer.eco-counter-tools.com/file/e1/da...,"48.89683,2.34521",Y2H21110997,https://filer.eco-counter-tools.com/file/e1/da...,https://filer.eco-counter-tools.com/file/e1/da...,https:,https://www.eco-visio.net/Photos/300021320,jpg,2023-01
893092,300021320-353293614,69 Boulevard Ornano N-S,300021320,69 Boulevard Ornano,0.0,2023-01-07T07:00:00+01:00,2021-11-30,https://filer.eco-counter-tools.com/file/e1/da...,"48.89683,2.34521",Y2H21110997,https://filer.eco-counter-tools.com/file/e1/da...,https://filer.eco-counter-tools.com/file/e1/da...,https:,https://www.eco-visio.net/Photos/300021320,jpg,2023-01
893093,300021320-353293614,69 Boulevard Ornano N-S,300021320,69 Boulevard Ornano,0.0,2023-01-07T08:00:00+01:00,2021-11-30,https://filer.eco-counter-tools.com/file/e1/da...,"48.89683,2.34521",Y2H21110997,https://filer.eco-counter-tools.com/file/e1/da...,https://filer.eco-counter-tools.com/file/e1/da...,https:,https://www.eco-visio.net/Photos/300021320,jpg,2023-01
893094,300021320-353293614,69 Boulevard Ornano N-S,300021320,69 Boulevard Ornano,0.0,2023-01-07T12:00:00+01:00,2021-11-30,https://filer.eco-counter-tools.com/file/e1/da...,"48.89683,2.34521",Y2H21110997,https://filer.eco-counter-tools.com/file/e1/da...,https://filer.eco-counter-tools.com/file/e1/da...,https:,https://www.eco-visio.net/Photos/300021320,jpg,2023-01


In [4]:
def missing(df,detail=True):
    total = 0
    for col in df.columns:
        miss = df[col].isnull().sum()
        pct = df[col].isna().mean() * 100
        total += miss
        if miss != 0:
            if pct>10: color=Fore.RED
            else: color=Fore.YELLOW
            print(color+'{} => {} [{}%]'.format(col, miss, round(pct, 2)))
        
        elif (total == 0) and(detail):
            print(Fore.GREEN+'{} => no missing values [{}%]'.format(col, 0))
        total=0

In [5]:
missing(df)

id_compteur => no missing values [0%]
nom_compteur => no missing values [0%]
id => no missing values [0%]
name => no missing values [0%]
sum_counts => no missing values [0%]
date => no missing values [0%]
installation_date => no missing values [0%]
url_photos_n1 => 9668 [1.08%]
coordinates => no missing values [0%]
counter => 18496 [2.07%]
photos => 9668 [1.08%]
test_lien_vers_photos_du_site_de_comptage_ => 9668 [1.08%]
id_photo_1 => 9668 [1.08%]
url_sites => no missing values [0%]
type_dimage => 9668 [1.08%]
mois_annee_comptage => no missing values [0%]


Manipulation du dataframe afin de déterminer le nombre de passage par jour à chaque point de comptage

In [6]:
def comptage_by_date(df):
    df['day'] = df['date'].str[:10] 
    
    #On fait la somme des velos comptés par jour à chaque point de comptage
    result = df.groupby(['id_compteur', 'day'])['sum_counts'].sum()
    
    #Ajout des résultats dans une nouvelle colonne afin de garder l'ensemble des informations initiales
    df = pd.merge(df, result, on=['id_compteur', 'day'], how='left') 
    df.rename(columns = {'sum_counts_x':'sum_counts', 'sum_counts_y':'sum_by_day'}, inplace = True)
    
    df['month'] = df['date'].str[:7] 
    
    #On fait la somme des velos comptés par mois à chaque point de comptage
    result = df.groupby(['id_compteur', 'month'])['sum_counts'].sum()
    
    #Ajout des résultats dans une nouvelle colonne afin de garder l'ensemble des informations initiales
    df = pd.merge(df, result, on=['id_compteur', 'month'], how='left') 
    df.rename(columns = {'sum_counts_x':'sum_counts', 'sum_counts_y':'sum_by_month'}, inplace = True)
    
    df['year'] = df['date'].str[:4] 
    
    #On fait la somme des velos comptés par année à chaque point de comptage
    result = df.groupby(['id_compteur', 'year'])['sum_counts'].sum()
    
    #Ajout des résultats dans une nouvelle colonne afin de garder l'ensemble des informations initiales
    df = pd.merge(df, result, on=['id_compteur', 'year'], how='left') 
    df.rename(columns = {'sum_counts_x':'sum_counts', 'sum_counts_y':'sum_by_year'}, inplace = True)
    
    return df

In [7]:
df = comptage_by_date(df)
df

,id_compteur,nom_compteur,id,name,sum_counts,date,installation_date,url_photos_n1,coordinates,counter,...,id_photo_1,url_sites,type_dimage,mois_annee_comptage,day,sum_by_day,month,sum_by_month,year,sum_by_year
0,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,0.0,2021-12-01T03:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/10/6d...,"48.83504,2.33314",X2H20012081,...,https:,https://www.eco-visio.net/Photos/100003096,jpg,2021-12,2021-12-01,1236.0,2021-12,27184.0,2021,27184.0
1,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,2.0,2021-12-01T04:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/10/6d...,"48.83504,2.33314",X2H20012081,...,https:,https://www.eco-visio.net/Photos/100003096,jpg,2021-12,2021-12-01,1236.0,2021-12,27184.0,2021,27184.0
2,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,0.0,2021-12-01T05:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/10/6d...,"48.83504,2.33314",X2H20012081,...,https:,https://www.eco-visio.net/Photos/100003096,jpg,2021-12,2021-12-01,1236.0,2021-12,27184.0,2021,27184.0
3,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,213.0,2021-12-01T08:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/10/6d...,"48.83504,2.33314",X2H20012081,...,https:,https://www.eco-visio.net/Photos/100003096,jpg,2021-12,2021-12-01,1236.0,2021-12,27184.0,2021,27184.0
4,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,84.0,2021-12-01T07:00:00+01:00,2012-02-22,https://filer.eco-counter-tools.com/file/10/6d...,"48.83504,2.33314",X2H20012081,...,https:,https://www.eco-visio.net/Photos/100003096,jpg,2021-12,2021-12-01,1236.0,2021-12,27184.0,2021,27184.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893091,300021320-353293614,69 Boulevard Ornano N-S,300021320,69 Boulevard Ornano,0.0,2023-01-07T05:00:00+01:00,2021-11-30,https://filer.eco-counter-tools.com/file/e1/da...,"48.89683,2.34521",Y2H21110997,...,https:,https://www.eco-visio.net/Photos/300021320,jpg,2023-01,2023-01-07,0.0,2023-01,0.0,2023,0.0
893092,300021320-353293614,69 Boulevard Ornano N-S,300021320,69 Boulevard Ornano,0.0,2023-01-07T07:00:00+01:00,2021-11-30,https://filer.eco-counter-tools.com/file/e1/da...,"48.89683,2.34521",Y2H21110997,...,https:,https://www.eco-visio.net/Photos/300021320,jpg,2023-01,2023-01-07,0.0,2023-01,0.0,2023,0.0
893093,300021320-353293614,69 Boulevard Ornano N-S,300021320,69 Boulevard Ornano,0.0,2023-01-07T08:00:00+01:00,2021-11-30,https://filer.eco-counter-tools.com/file/e1/da...,"48.89683,2.34521",Y2H21110997,...,https:,https://www.eco-visio.net/Photos/300021320,jpg,2023-01,2023-01-07,0.0,2023-01,0.0,2023,0.0
893094,300021320-353293614,69 Boulevard Ornano N-S,300021320,69 Boulevard Ornano,0.0,2023-01-07T12:00:00+01:00,2021-11-30,https://filer.eco-counter-tools.com/file/e1/da...,"48.89683,2.34521",Y2H21110997,...,https:,https://www.eco-visio.net/Photos/300021320,jpg,2023-01,2023-01-07,0.0,2023-01,0.0,2023,0.0


Manipulation des coordonnées GPS

In [8]:
def handle_coordinates(df):
    #format colonne coordinates : 'latitude,longitude'
    df['latitude'] = df['coordinates'].apply(lambda x: x.split(',')[0]).astype('float')
    df['longitude'] = df['coordinates'].apply(lambda x: x.split(',')[1]).astype('float')
    
    return df

In [9]:
df = handle_coordinates(df)

Statistiques par compteur

In [10]:
def statistic_dataframe(df):
    
    day_mean = df.groupby(['id_compteur'])['sum_by_day'].mean().astype('int32')
    month_mean = df.groupby(['id_compteur'])['sum_by_month'].mean().astype('int32')
    year_mean = df.groupby(['id_compteur'])['sum_by_year'].mean().astype('int32')
    
    new_df = pd.merge(day_mean, month_mean, on=['id_compteur'], how='left') 
    new_df = pd.merge(new_df, year_mean, on=['id_compteur'], how='left') 
    
    
    lat = df.groupby(['id_compteur'])['latitude'].mean() #same value for all value, it doesn't change anything
    long = df.groupby(['id_compteur'])['longitude'].mean()
    
    new_df = pd.merge(new_df, lat, on=['id_compteur'], how='left')
    new_df = pd.merge(new_df, long, on=['id_compteur'], how='left')
    
    adresse = df.groupby(['id_compteur'])['name'].unique().map(lambda x: x[0])
    new_df = pd.merge(new_df, adresse, on=['id_compteur'], how='left')
    
    date_install = df.groupby(['id_compteur'])['installation_date'].unique().map(lambda x: x[0])
    new_df = pd.merge(new_df, date_install, on=['id_compteur'], how='left')
        
    new_df.rename(columns = {'sum_by_day':'mean_day', 'sum_by_month':'mean_month', 'sum_by_year':'mean_year', 'name':'adresse'}, inplace = True)
    
    return new_df

In [11]:
stat_df = statistic_dataframe(df)
stat_df.reset_index(inplace = True)
stat_df

,id_compteur,mean_day,mean_month,mean_year,latitude,longitude,adresse,installation_date
0,100003096-353242251,1236,37175,421519,48.835040,2.333140,97 avenue Denfert Rochereau,2012-02-22
1,100003097-101003097,1519,45375,333404,48.877730,2.350600,105 rue La Fayette,2012-02-13
2,100003098-101003098,1590,47871,542737,48.835050,2.333330,106 avenue Denfert Rochereau,2012-02-22
3,100003099-353242239,0,1,17,48.878130,2.349700,100 rue La Fayette,2012-02-22
4,100006300-101006300,1577,47518,538770,48.843435,2.383378,135 avenue Daumesnil,2013-01-19
...,...,...,...,...,...,...,...,...
94,100065336-104065336,3330,100127,1156706,48.860528,2.388364,35 boulevard de Ménilmontant,2020-11-18
95,300014702-353245971,1724,49873,491024,48.839850,2.302020,254 rue de Vaugirard,2020-11-29
96,300014702-353245972,1429,41335,409066,48.839850,2.302020,254 rue de Vaugirard,2020-11-29
97,300021312-353293580,3016,90841,1027136,48.851131,2.345678,30 rue Saint Jacques,2021-11-30


Affichage lieu de comptage sur une map via la librairie Folium

In [22]:
def make_map(df):
    #Create the base Map
    m = folium.Map(location=[48.866667,2.333333], tiles='OpenStreetMap', zoom_start=12)

    markerCluster = MarkerCluster().add_to(m)
    for i, row in df.iterrows():
        lat = df.at[i,'latitude']
        lng = df.at[i,'longitude']

        compteur = df.at[i,'id_compteur']

        sum_day = df.at[i,'mean_day']
        if sum_day > 1236:
            if sum_day > 1962:
                color = 'red'
            else:
                color = 'orange'
        else:
            if sum_day < 879:
                color = 'green'
            else:
                color = 'yellow'

        folium.Marker(location=[lat, lng], popup=sum_day, icon=folium.Icon(color=color)).add_to(markerCluster)

    m.save("bike_counts_by_day.html")

In [23]:
make_map(stat_df)

C:\Users\coco8\AppData\Local\Temp\ipykernel_83368\800563660.py:24: UserWarning: color argument of Icon should be one of: {'darkpurple', 'cadetblue', 'gray', 'blue', 'white', 'lightgray', 'pink', 'lightgreen', 'beige', 'orange', 'darkgreen', 'red', 'green', 'darkred', 'purple', 'darkblue', 'lightblue', 'black', 'lightred'}.
  folium.Marker(location=[lat, lng], popup=sum_day, icon=folium.Icon(color=color)).add_to(markerCluster)
